[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/racousin/rag_attack/blob/main/rag_attack_partie2_agentic.ipynb)

# RAG Attack - Partie 2: Agentic RAG avec LangGraph

## Introduction aux Systèmes Agentiques

Bienvenue dans cette formation sur les systèmes RAG agentiques ! Après avoir vu les bases du RAG (embeddings, vector search) dans la partie 1, nous allons maintenant explorer comment rendre ces systèmes intelligents et autonomes.

### Qu'est-ce qu'un Agent ?

Un **agent** est un système qui peut :
- **Raisonner** sur une tâche
- **Planifier** une séquence d'actions
- **Utiliser des outils** pour accomplir ses objectifs
- **S'adapter** en fonction des résultats obtenus

### Pourquoi LangGraph ?

LangGraph nous permet de créer des workflows complexes sous forme de graphes d'états, parfait pour orchestrer des agents intelligents.

## Setup et Configuration

Commençons par installer les dépendances nécessaires et configurer notre environnement Azure.

In [24]:
# Installation et clonage du repository
# %%capture
!git clone https://github.com/racousin/rag_attack.git rag_attack_repo
!pip install -e rag_attack_repo

Cloning into 'rag_attack_repo'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 134 (delta 66), reused 106 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (134/134), 166.30 KiB | 126.00 KiB/s, done.
Resolving deltas: 100% (66/66), done.
Obtaining file:///Users/raphaelcousin/rag_attack/rag_attack_repo
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for rag-attack (pyproject.toml) ... done
  Created wheel for rag-attack: filename=rag_attack-0.1.0-0.editable-py3-none-any.whl size=2894 sha256=8205bb9f64c8f950f61aec6f21d73a6d6e4506eeff0f299b22f7f26b3c6b09e3
  Stored in directory: /private/var/folders/n1/bdjyqqwn5t5cglz8mdg_13900000gn/T/pip-ephem-wheel-cache-8hvi7mug/wheels/1c/97/e1/ceceacbe28e29e72b3

In [25]:
# Import des bibliothèques nécessaires
import sys
import os
from pathlib import Path

# Ajouter le package rag_attack au path
sys.path.insert(0, str(Path.cwd()))

# Imports de base
import json
from typing import List, Dict, Any
from IPython.display import display, Markdown, HTML
import warnings
warnings.filterwarnings('ignore')

print("✅ Environnement configuré avec succès !")

✅ Environnement configuré avec succès !


In [ ]:
# Configuration Azure - À compléter par l'étudiant
# Copiez-collez vos identifiants Azure fournis par votre instructeur

config = {
  "search_endpoint": "",
  "search_key": "",
  "sql_server": "",
  "sql_database": "",
  "sql_username": "",
  "sql_password": "",
  "api_base_url": "",
  "openai_endpoint": "",
  "openai_key": "",
  "chat_deployment": ""
}


# Vérification que la configuration est complète
missing_fields = [key for key, value in config.items() if not value]

if missing_fields:
    print("⚠️ Configuration incomplète. Veuillez remplir les champs suivants:")
    for field in missing_fields:
        print(f"   - {field}")
    print("\n💡 Conseil: Copiez les valeurs depuis le fichier azure_credentials.json fourni")
else:
    # IMPORTANT: Set the global configuration for all tools
    from rag_attack import set_config
    set_config(config)
    
    print("✅ Configuration Azure chargée avec succès !")
    print(f"   - Azure OpenAI Endpoint: {config['openai_endpoint']}")
    print(f"   - Azure Search Endpoint: {config['search_endpoint']}")
    print(f"   - SQL Server: {config['sql_server']}")
    print("\n✅ Configuration globale définie pour tous les outils !")

## Chapitre 1: Agent Simple avec Outils RAG

![LangGraph](images/langgraph.png)

### 1.1 Concept de Base

Un agent simple suit ce cycle :
1. **Recevoir** une question
2. **Décider** si un outil est nécessaire
3. **Utiliser** le bon outil
4. **Formuler** une réponse

### 1.2 Implémentation

In [27]:
# Import des outils RAG
from rag_attack import SimpleToolAgent, azure_search_tool, azure_vector_search_tool

# Création d'un agent simple avec l'outil de recherche Azure
simple_agent = SimpleToolAgent(
    config=config,
    tools=[azure_search_tool],
    system_prompt="""Tu es un assistant expert en recherche d'information sur VéloCorp.
    Utilise l'outil de recherche Azure pour trouver des informations pertinentes.
    Réponds en français de manière claire et structurée."""
)

print("✅ Agent simple créé avec l'outil de recherche Azure")

✅ Agent simple créé avec l'outil de recherche Azure


In [28]:
# Test de l'agent simple
question = "Quels sont les modèles de vélos électriques proposés par VéloCorp ? Explique dans quel document tu as eu ces informations"

print(f"❓ Question: {question}\n")
print("🤖 Réponse de l'agent:\n")
print("-" * 50)

response = simple_agent.invoke(question)
display(Markdown(response))

❓ Question: Quels sont les modèles de vélos électriques proposés par VéloCorp ? Explique dans quel document tu as eu ces informations

🤖 Réponse de l'agent:

--------------------------------------------------


Il semble que les recherches effectuées ne permettent pas de trouver des informations spécifiques sur les modèles de vélos électriques proposés par VéloCorp. Les documents disponibles ne contiennent pas de détails sur les modèles de vélos électriques.

Je recommande de consulter directement le site web de VéloCorp ou de contacter leur service client pour obtenir des informations précises sur les modèles de vélos électriques qu'ils proposent.

### 1.3 Visualisation du Flux de l'Agent

Voyons comment l'agent traite la requête étape par étape :

In [29]:
# Streaming pour voir le processus en temps réel
def visualize_agent_stream(agent, question):
    """Visualise le flux de traitement de l'agent"""
    print(f"📝 Question: {question}\n")
    
    for i, chunk in enumerate(agent.stream(question)):
        print(f"\n🔄 Étape {i+1}:")
        
        # Afficher le type de nœud
        for node, data in chunk.items():
            print(f"   📍 Nœud: {node}")
            
            if 'messages' in data:
                last_msg = data['messages'][-1]
                
                # Afficher le type de message
                if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                    print(f"   🔧 Appel d'outil détecté: {last_msg.tool_calls[0]['name']}")
                elif hasattr(last_msg, 'content'):
                    content_preview = str(last_msg.content)[:200]
                    print(f"   💬 Contenu: {content_preview}...")

# Test avec visualisation
visualize_agent_stream(simple_agent, "Quelle est la politique de garantie de VéloCorp ?")

📝 Question: Quelle est la politique de garantie de VéloCorp ?


🔄 Étape 1:
   📍 Nœud: agent
   🔧 Appel d'outil détecté: azure_search_tool

🔄 Étape 2:
   📍 Nœud: tools
   💬 Contenu: Result 1:
  Title: No Title
  Category: Unknown
  Source: Unknown
  Content: FAQ - Garantie

Foire aux questions concernant garantie. Retrouvez ici les réponses aux questions les plus fréquemment posé...

🔄 Étape 3:
   📍 Nœud: agent
   💬 Contenu: La politique de garantie de VéloCorp est la suivante :

- **Durée de la garantie** :
  - La garantie du cadre est de 2 ans.
  - Les composants sont garantis 1 an.

- **Ce que couvre la garantie** :
  ...


## Chapitre 2: Agent ReAct (Reasoning and Acting)

### 2.1 Le Framework ReAct

ReAct est un paradigme puissant qui alterne entre :
- **Thought** (Réflexion) : L'agent raisonne sur ce qu'il doit faire
- **Action** : L'agent exécute une action (utilise un outil)
- **Observation** : L'agent observe le résultat

Cette approche rend le processus de décision **transparent** et **débogable**.

In [30]:
# Import de l'agent ReAct
from rag_attack.agents import ReActAgent
from rag_attack.tools import azure_search_tool, sql_query_tool, get_database_schema

# Création d'un agent ReAct
react_agent = ReActAgent(
    tools=[
        azure_search_tool,
    ],
    max_iterations=5
)

print("✅ Agent ReAct")

✅ Agent ReAct


In [31]:
# Test de l'agent ReAct avec une question complexe
complex_question = """Trouve-moi les informations sur les vélos électriques de VéloCorp, quels sont les retours de satisfaction clients?"""

print(f"❓ Question complexe: {complex_question}\n")
print("="*60)

# Obtenir la trace complète du raisonnement
trace = react_agent.get_reasoning_trace(complex_question)

# Afficher le processus de raisonnement
print("\n🧠 PROCESSUS DE RAISONNEMENT:\n")
for i, (thought, obs) in enumerate(zip(trace['reasoning'], trace['observations']), 1):
    print(f"\n{'='*60}")
    print(f"Étape {i}:")
    print(f"\n💭 Réflexion: {thought}")
    print(f"\n👁️ Observation: {obs}")

print(f"\n{'='*60}")
print(f"\n✅ RÉPONSE FINALE:\n{trace['final_answer']}")

❓ Question complexe: Trouve-moi les informations sur les vélos électriques de VéloCorp, quels sont les retours de satisfaction clients?


############################################################
🤖 ReAct Agent Starting (Trace Mode)
############################################################
❓ Question: Trouve-moi les informations sur les vélos électriques de VéloCorp, quels sont les retours de satisfaction clients?
🔧 Available tools: azure_search_tool
🔄 Max iterations: 5
############################################################

🧠 STEP 1: REASONING
⏳ Thinking...

💭 Thought: 1. What information do I still need to answer the task?
I need detailed information about the electric bikes offered by VéloCorp, including customer satisfaction feedback and reviews.

2. Which specific tool should I use next to get that information?
I should use the azure_search_tool to search for documents related to VéloCorp's electric bikes and customer satisfaction.

3. What exact input should I provide 

GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT

### 2.2 Comparaison: Agent Simple vs ReAct

Comparons les deux approches sur la même question :

In [ ]:
# Question de test
test_question = "Quel est le chiffre d'affaires de VéloCorp et combien de clients ont-ils ?"

print("📊 COMPARAISON DES APPROCHES\n")
print("="*60)

# Test avec l'agent simple
print("\n1️⃣ AGENT SIMPLE:")
print("-"*40)
simple_response = simple_agent.invoke(test_question)
print(simple_response[:500] + "..." if len(simple_response) > 500 else simple_response)

# Test avec l'agent ReAct
print("\n2️⃣ AGENT REACT:")
print("-"*40)
react_response = react_agent.invoke(test_question)
print(react_response[:500] + "..." if len(react_response) > 500 else react_response)

print("\n" + "="*60)
print("\n💡 Observation: L'agent ReAct fournit un raisonnement plus structuré !")

## Chapitre 3: Agent Multi-Outils

### 3.1 Intégration de Multiples Sources de Données

Un agent vraiment utile doit pouvoir orchestrer plusieurs outils :
- **Recherche documentaire** (Azure Search)
- **Base de données** (SQL)
- **APIs externes** (CRM, Weather, etc.)
- **Web** (recherche internet)

In [32]:
# Import de tous les outils disponibles
from rag_attack import (
    SimpleToolAgent,
    azure_search_tool,
    sql_query_tool,
    sql_table_info,
    crm_opportunities_tool,
    weather_api_tool,
    web_search_tool
)

# Création d'un super-agent avec tous les outils
super_agent = SimpleToolAgent(
    config=config,
    tools=[
        azure_search_tool,
        sql_query_tool,
        sql_table_info,
        crm_opportunities_tool,
        weather_api_tool,
        web_search_tool
    ],
    system_prompt="""Tu es un assistant expert multi-domaines pour VéloCorp.
    Tu peux:
    - Rechercher dans la documentation interne
    - Interroger la base de données
    - Accéder au CRM pour les opportunités commerciales
    - Obtenir la météo (utile pour planifier des événements vélo)
    - Rechercher sur internet pour des informations complémentaires
    
    Utilise intelligemment ces outils pour fournir des réponses complètes et pertinentes."""
)

print("✅ Super-agent créé avec accès à:")
print("   - Azure Search")
print("   - SQL Database")
print("   - CRM Opportunities")
print("   - Weather API")
print("   - Web Search")

✅ Super-agent créé avec accès à:
   - Azure Search
   - SQL Database
   - CRM Opportunities
   - Weather API
   - Web Search


In [34]:
# Test du super-agent avec une requête complexe nécessitant plusieurs outils
multi_tool_question = """Je prépare une présentation pour les investisseurs. 
J'ai besoin de:
1. Les dernières opportunités commerciales du CRM
2. Les statistiques de ventes depuis la base de données
3. La météo à Paris pour notre événement de lancement la semaine prochaine
"""

print(f"📋 Requête multi-outils:\n{multi_tool_question}\n")
print("="*60)
print("\n🤖 Traitement en cours...\n")

response = super_agent.invoke(multi_tool_question)
display(Markdown(response))

📋 Requête multi-outils:
Je prépare une présentation pour les investisseurs. 
J'ai besoin de:
1. Les dernières opportunités commerciales du CRM
2. Les statistiques de ventes depuis la base de données
3. La météo à Paris pour notre événement de lancement la semaine prochaine



🤖 Traitement en cours...



Je n'ai pas pu récupérer les informations des opportunités commerciales du CRM ni les statistiques de ventes en raison des erreurs suivantes :

1. **Opportunités commerciales du CRM**: Erreur 404 (Not Found) lors de la requête.
2. **Statistiques de ventes**: Erreur SQL indiquant que l'objet 'sales_statistics' est invalide.

Pour résoudre ces problèmes, pourriez-vous vérifier :
- L'URL et l'accessibilité de l'API CRM.
- Le nom exact de la table contenant les statistiques de ventes dans la base de données.

En attendant, voici les informations météo pour Paris :

### Météo à Paris pour notre événement de lancement la semaine prochaine
- **Description**: Partiellement nuageux
- **Température**: 10°C
- **Humidité**: 76%
- **Vitesse du vent**: 15 km/h

Si vous avez des informations supplémentaires ou des corrections, je peux réessayer de récupérer les données nécessaires.

### 3.2 Cas d'Usage: Analyse Business Complète

In [ ]:
# Fonction d'aide pour une analyse business complète
def business_analysis(company_context: str, agent: SimpleToolAgent):
    """Effectue une analyse business complète"""
    
    analyses = {
        "📊 Analyse des Ventes": "Quelles sont les tendances de ventes récentes dans la base de données ?",
        "🎯 Opportunités CRM": "Quelles sont les opportunités commerciales à fort potentiel ?",
        "📚 Documentation Produit": "Quels sont nos principaux produits et leurs caractéristiques ?",
        "🌍 Analyse Marché": "Recherche sur internet les dernières tendances du marché du vélo électrique",
    }
    
    results = {}
    
    for title, question in analyses.items():
        print(f"\n{title}")
        print("-" * 40)
        
        full_question = f"{company_context} {question}"
        response = agent.invoke(full_question)
        
        results[title] = response
        print(response[:300] + "..." if len(response) > 300 else response)
    
    return results

# Exécution de l'analyse
context = "Pour VéloCorp, entreprise de vélos électriques premium,"
print("🔍 ANALYSE BUSINESS COMPLÈTE")
print("="*60)

analysis_results = business_analysis(context, super_agent)

## Chapitre 4: Architecture Planner

### 4.1 Le Concept de Planification Hiérarchique

Pour des tâches vraiment complexes, nous avons besoin d'un **planner** qui peut :
1. **Décomposer** une tâche complexe en sous-tâches
2. **Gérer les dépendances** entre les tâches
3. **Orchestrer l'exécution** dans le bon ordre
4. **S'adapter** si quelque chose échoue

In [ ]:
# Import du planner hiérarchique
from rag_attack.planners import HierarchicalPlanner

# Création du planner avec tous les outils
planner = HierarchicalPlanner(
    tools=[
        azure_search_tool,
        sql_query_tool,
        get_database_schema,
        crm_opportunities_tool,
        web_search_tool
    ],
    max_steps=8
)

print("✅ Planner hiérarchique créé")

In [ ]:
# Test du planner avec un objectif complexe
complex_objective = """Prépare un rapport complet pour le conseil d'administration incluant:
1. Analyse de la performance commerciale actuelle
2. Étude de marché sur les tendances du vélo électrique
3. Recommandations stratégiques basées sur les données
"""

print(f"🎯 Objectif complexe:\n{complex_objective}\n")
print("="*60)
print("\n⚙️ Exécution du plan...\n")

# Exécution avec le planner
result = planner.execute(complex_objective)

# Affichage du plan
print("\n📋 PLAN GÉNÉRÉ:")
print("-" * 40)
for step in result['plan']:
    status_emoji = "✅" if step['status'] == "completed" else "⏳"
    print(f"{status_emoji} Étape {step['step_id']}: {step['description']}")
    if step['tool']:
        print(f"   🔧 Outil: {step['tool']}")
    if step['dependencies']:
        print(f"   🔗 Dépend de: {step['dependencies']}")

# Affichage du résultat final
print("\n" + "="*60)
print("\n📝 RÉSULTAT FINAL:")
# display(Markdown(result['final_result']))

## Chapitre 6: Bonnes Pratiques et Optimisations

### 6.1 Gestion des Erreurs et Fallback

In [ ]:
# Agent robuste avec gestion d'erreurs
from langchain_core.tools import tool
import time

@tool
def safe_search_with_retry(query: str, max_retries: int = 3) -> str:
    """Recherche avec gestion d'erreurs et retry automatique"""
    
    for attempt in range(max_retries):
        try:
            # Tentative de recherche
            result = azure_search_tool.invoke(query)
            
            if result and "No results" not in result:
                return result
            
            # Si pas de résultats, essayer avec une requête modifiée
            if attempt < max_retries - 1:
                # Simplifier la requête
                words = query.split()
                if len(words) > 2:
                    query = " ".join(words[:2])  # Garder seulement les 2 premiers mots
                time.sleep(1)  # Petit délai entre les tentatives
                
        except Exception as e:
            if attempt == max_retries - 1:
                return f"Erreur après {max_retries} tentatives: {str(e)}"
            time.sleep(2 ** attempt)  # Backoff exponentiel
    
    return "Aucun résultat trouvé après plusieurs tentatives"

# Test de la recherche robuste
test_queries = [
    "vélos électriques premium haute performance 2024",  # Requête complexe
    "xyzabc123nonexistent",  # Requête qui ne donnera pas de résultats
]

print("🛡️ TEST DE ROBUSTESSE\n")
print("="*60)

for query in test_queries:
    print(f"\n🔍 Recherche: {query}")
    result = safe_search_with_retry(query)
    print(f"📝 Résultat: {result[:200]}..." if len(result) > 200 else f"📝 Résultat: {result}")

### 6.2 Monitoring et Métriques de Performance

In [ ]:
# Système de monitoring pour les agents
import time
from datetime import datetime
from typing import Dict, Any

class AgentMonitor:
    """Monitore les performances des agents"""
    
    def __init__(self):
        self.metrics = []
    
    def track_execution(self, agent_name: str, question: str, agent_func):
        """Track l'exécution d'un agent"""
        start_time = time.time()
        
        try:
            result = agent_func(question)
            success = True
            error = None
        except Exception as e:
            result = None
            success = False
            error = str(e)
        
        execution_time = time.time() - start_time
        
        # Enregistrer les métriques
        self.metrics.append({
            "timestamp": datetime.now(),
            "agent": agent_name,
            "question_length": len(question),
            "execution_time": execution_time,
            "success": success,
            "error": error,
            "response_length": len(result) if result else 0
        })
        
        return result
    
    def get_statistics(self) -> Dict[str, Any]:
        """Calcule les statistiques de performance"""
        if not self.metrics:
            return {"message": "Aucune métrique disponible"}
        
        import pandas as pd
        df = pd.DataFrame(self.metrics)
        
        stats = {
            "total_executions": len(df),
            "success_rate": (df['success'].sum() / len(df)) * 100,
            "avg_execution_time": df['execution_time'].mean(),
            "max_execution_time": df['execution_time'].max(),
            "min_execution_time": df['execution_time'].min(),
            "avg_response_length": df['response_length'].mean()
        }
        
        # Statistiques par agent
        agent_stats = {}
        for agent in df['agent'].unique():
            agent_df = df[df['agent'] == agent]
            agent_stats[agent] = {
                "executions": len(agent_df),
                "avg_time": agent_df['execution_time'].mean(),
                "success_rate": (agent_df['success'].sum() / len(agent_df)) * 100
            }
        
        stats["by_agent"] = agent_stats
        
        return stats

# Test du monitoring
monitor = AgentMonitor()

# Exécuter plusieurs requêtes avec monitoring
test_questions = [
    "Quels sont les vélos électriques ?",
    "Analyse les ventes du dernier trimestre",
    "Trouve les opportunités CRM importantes"
]

print("📊 MONITORING DES AGENTS\n")
print("="*60)

for q in test_questions:
    print(f"\n⏱️ Exécution: {q[:50]}...")
    
    # Test avec agent simple
    result = monitor.track_execution(
        "SimpleAgent",
        q,
        lambda x: simple_agent.invoke(x)
    )
    
    # Test avec agent ReAct
    result = monitor.track_execution(
        "ReActAgent",
        q,
        lambda x: react_agent.invoke(x)
    )

# Afficher les statistiques
stats = monitor.get_statistics()

print("\n" + "="*60)
print("\n📈 STATISTIQUES DE PERFORMANCE:\n")
print(f"Total d'exécutions: {stats['total_executions']}")
print(f"Taux de succès: {stats['success_rate']:.1f}%")
print(f"Temps moyen: {stats['avg_execution_time']:.2f}s")
print(f"Temps min/max: {stats['min_execution_time']:.2f}s / {stats['max_execution_time']:.2f}s")

print("\n📊 Par Agent:")
for agent, agent_stats in stats['by_agent'].items():
    print(f"\n  {agent}:")
    print(f"    - Exécutions: {agent_stats['executions']}")
    print(f"    - Temps moyen: {agent_stats['avg_time']:.2f}s")
    print(f"    - Taux de succès: {agent_stats['success_rate']:.1f}%")

## Conclusion et Ressources

### 🎯 Ce que nous avons appris

1. **Agents Simples** : Comment créer des agents qui utilisent des outils pour répondre aux questions
2. **Framework ReAct** : L'importance de la transparence dans le raisonnement
3. **Multi-Outils** : Orchestration de plusieurs sources de données
4. **Planification** : Décomposition de tâches complexes
5. **Robustesse** : Gestion d'erreurs et monitoring

### 🚀 Prochaines Étapes

1. **Expérimenter** avec différentes combinaisons d'outils
2. **Personnaliser** les prompts système pour votre domaine
3. **Intégrer** ces agents dans vos applications
4. **Optimiser** les performances avec du caching et de la parallélisation

### 💡 Points Clés à Retenir

1. **Les agents transforment le RAG** de passif à actif
2. **La transparence** du raisonnement est cruciale
3. **L'orchestration** d'outils multiplie les capacités
4. **La planification** permet de résoudre des problèmes complexes
5. **Le monitoring** est essentiel en production

## Exercices Pratiques

### Exercice 1: Créer votre propre outil

Créez un outil personnalisé qui calcule des métriques business spécifiques.

In [ ]:
# TODO: Créez votre propre outil ici
# Exemple: Un outil qui calcule le ROI d'une campagne marketing

@tool
def calculate_roi(investment: float, revenue: float) -> str:
    """Calcule le ROI (Return on Investment)
    
    Args:
        investment: Montant investi
        revenue: Revenue généré
    
    Returns:
        ROI en pourcentage
    """
    # Votre implémentation ici
    pass

### Exercice 2: Agent Spécialisé

Créez un agent spécialisé pour un cas d'usage spécifique de votre choix.

In [ ]:
# TODO: Créez votre agent spécialisé
# Exemple: Un agent pour le support client

# Votre code ici

### Exercice 3: Plan Complexe

Utilisez le planner pour résoudre un problème business réel de votre organisation.

In [ ]:
# TODO: Définissez un objectif complexe et utilisez le planner
# Exemple: "Préparer le lancement d'un nouveau produit"

# Votre code ici

---

🎉 **Félicitations !** Vous maîtrisez maintenant les concepts avancés du RAG agentique avec LangGraph !

N'hésitez pas à expérimenter et adapter ces exemples à vos besoins spécifiques.